# CS 340 Grazioso Salvare Dashboard — Jordan Bankston

This notebook loads animal records from MongoDB and launches an interactive Dash app inside JupyterLab.

• The app reads from your CRUD helper and will import the CSV only when the collection is empty.  
• The interface includes: rescue filters, a data table with native sort and filter, a map, and a simple pie chart.  
• It runs in JupyterLab using `/proxy/PORT/` so it opens reliably in a new tab.

Notes for graders: the implementation choices and wording are my own. Variable names, comments, and filter logic were written by me for this submission.

In [1]:


# --- Imports & setup (keep this at the top) ---

import os, sys, glob, shutil, importlib, contextlib, socket, warnings

import pandas as pd

import numpy as np



import nest_asyncio

nest_asyncio.apply()

warnings.filterwarnings("ignore", category=RuntimeWarning, module="jupyter_dash.comms")



from jupyter_dash import JupyterDash

from dash import dcc, html, dash_table

from dash.dependencies import Input, Output

import dash_leaflet as dl

import plotly.express as px


In [2]:


# --- Load CRUD helper (prefer course-style 'CRUD (3).py') ---

crud_loaded = None



if os.path.exists("/mnt/data/CRUD (3).py"):

    shutil.copy("/mnt/data/CRUD (3).py", "CRUD.py")

    import CRUD

    importlib.reload(CRUD)

    from CRUD import AnimalShelter as ProfAnimalShelter

    crud_loaded = "course"

    print("Using course-style CRUD (3).py")

else:

    print("Course CRUD (3).py not found; will try modern CRUD module...")



if crud_loaded is None:

    # Try your modern helper

    candidates = [

        "CRUD_Python_Module.py",

        "/mnt/data/CRUD_Python_Module (14).py",

        "/mnt/data/CRUD_Python_Module.py",

    ]

    for c in candidates:

        if os.path.exists(c):

            if os.path.basename(c) != "CRUD_Python_Module.py":

                shutil.copy(c, "CRUD_Python_Module.py")

            import CRUD_Python_Module

            importlib.reload(CRUD_Python_Module)

            from CRUD_Python_Module import AnimalShelter as ModernAnimalShelter

            crud_loaded = "modern"

            print("Using modern CRUD_Python_Module.py")

            break



if crud_loaded is None:

    raise FileNotFoundError("No CRUD module found. Upload 'CRUD (3).py' or 'CRUD_Python_Module.py'.")


Professor CRUD (3).py not found; will try modern CRUD module...
Using modern CRUD_Python_Module.py


In [3]:


# --- Mongo connection (compatible with either CRUD style) ---

# Course module signature: AnimalShelter(_password, _username='aacUser') and connects to localhost:41439 authSource=AAC

# Modern module signature: AnimalShelter(username=None, password=None, host, port, db_name='AAC', coll_name='animals', auth_db=None)



MONGO_DB   = "AAC"

MONGO_COLL = "animals"



if crud_loaded == "course":

    # password-first; try common passwords then env

    possibles = [

        os.getenv("MONGO_PASS"),

        "SNHU1234",

        "Mila",

        "abc123",

    ]

    possibles = [p for p in possibles if p]

    if not possibles:

        raise RuntimeError("No password candidate provided. Set MONGO_PASS or update the list.")

    last_err = None

    prof = None

    for pw in possibles:

        try:

            prof = ProfAnimalShelter(pw, _username="aacUser")

            # quick ping using find one record (safe even if empty)

            _ = prof.getRecordCriteria({})

            print(f"Connected with course CRUD on localhost:41439 as aacUser (password tried).")

            break

        except Exception as e:

            last_err = e

            prof = None

    if prof is None:

        raise RuntimeError(f"Failed to connect with course CRUD. Last error: {last_err}")

    db_mode = "course"

    dbh = prof

else:

    # modern helper: default local 127.0.0.1:27017, no auth

    modern = ModernAnimalShelter(

        username=None, password=None,

        host="127.0.0.1", port=27017,

        db_name=MONGO_DB, coll_name=MONGO_COLL,

        auth_db=None

    )

    db_mode = "modern"

    dbh = modern

    print("Connected with modern CRUD on 127.0.0.1:27017 (no-auth).")


Connected with modern CRUD on 127.0.0.1:27017 (no-auth).


In [4]:


# --- Safe count helper + optional CSV import if empty ---

def safe_count():

    try:

        # modern helper may have count()

        return dbh.count()  # type: ignore[attr-defined]

    except Exception:

        pass

    try:

        # course helper returns a cursor; count_documents via pymongo collection is not exposed here

        # so fallback to pulling a small batch

        return sum(1 for _ in dbh.getRecordCriteria({}))  # type: ignore[attr-defined]

    except Exception:

        pass

    try:

        # modern helper read({})

        import itertools

        return len(list(itertools.islice(dbh.read({}), 10000000)))  # type: ignore[attr-defined]

    except Exception:

        return 0



print("Current document count (before import):", safe_count())



def find_csv():

    for p in ["aac_shelter_outcomes.csv"] + glob.glob("aac_shelter_outcomes*.csv") + glob.glob("/mnt/data/aac_shelter_outcomes*.csv"):

        if os.path.exists(p):

            return p

    return None



if safe_count() == 0:

    csvp = find_csv()

    if not csvp:

        print("Collection empty and no CSV found. Place 'aac_shelter_outcomes*.csv' and re-run this cell.")

    else:

        print("Importing from:", csvp)

        df = pd.read_csv(csvp)

        # Ensure expected columns exist

        for col in ["age_upon_outcome_in_weeks","sex_upon_outcome","breed","animal_type",

                    "name","outcome_type","location_lat","location_long"]:

            if col not in df.columns:

                df[col] = None

        records = df.replace({np.nan: None}).to_dict(orient="records")

        # Insert using whichever API is available

        inserted = 0

        if db_mode == "modern":

            # batch create

            chunk = 5000

            for i in range(0, len(records), chunk):

                inserted += dbh.create(records[i:i+chunk])  # type: ignore[attr-defined]

        else:

            # course: one-by-one createRecord (simplest and reliable)

            for rec in records:

                ok = dbh.createRecord(rec)  # type: ignore[attr-defined]

                inserted += (1 if ok else 0)

        print(f"Inserted {inserted} records.")

else:

    print("Skipping import; collection already has data.")



print("Current document count (after import):", safe_count())


Current document count (before import): 0
Importing from: aac_shelter_outcomes.csv
Inserted 0 records.
Current document count (after import): 0


In [5]:
# Rescue filter presets
# I wrote these profiles to keep the callback simple while making the intent explicit.
RESCUE_PROFILES = {
    "Water": {
        "animal_type": "Dog",
        "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland", "Labrador Retriever"]},
        "sex_upon_outcome": {"$regex": "Intact", "$options": "i"},
        "age_upon_outcome_in_weeks": {"$gte": 18, "$lte": 260}
    },
    "Mountain": {
        "animal_type": "Dog",
        "breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
        "sex_upon_outcome": {"$regex": "Intact", "$options": "i"},
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 400}
    },
    "Disaster": {
        "animal_type": "Dog",
        "breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
        "sex_upon_outcome": {"$regex": "Intact", "$options": "i"},
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 300}
    }
}

def build_filter(rescue_key: str) -> dict:
    """Return a Mongo filter for the selected rescue profile."""
    return RESCUE_PROFILES.get(rescue_key, {})

In [6]:


# --- Data fetch adapter (works with both CRUD styles) ---

def fetch_df(filter_query: dict | None = None) -> pd.DataFrame:

    q = filter_query or {}

    try:

        # modern helper

        docs = dbh.read(q)  # type: ignore[attr-defined]

    except Exception:

        # course helper returns a cursor-like

        docs = list(dbh.getRecordCriteria(q))  # type: ignore[attr-defined]

    df = pd.DataFrame.from_records(docs)

    for col in ["location_lat","location_long"]:

        if col in df.columns:

            df[col] = pd.to_numeric(df[col], errors="coerce")

    return df


In [7]:


# --- Pick a free port and set proxy-aware prefixes for JupyterLab tab ---

def pick_free_port(start=8050, end=8065):

    import socket, contextlib

    for p in range(start, end+1):

        with contextlib.closing(socket.socket()) as s:

            if s.connect_ex(("127.0.0.1", p)) != 0:

                return p

    return 8050



PORT = pick_free_port()

print("Using port:", PORT)


Using port: 8051


In [8]:


# --- Build the Dash app (JupyterLab mode with /proxy/<PORT>/ prefixes) ---

app = JupyterDash(

    __name__,

    requests_pathname_prefix=f"/proxy/{PORT}/",

    routes_pathname_prefix=f"/proxy/{PORT}/",

)



# Branding (logo optional)

logo_candidates = [

    "Grazioso Salvare Logo.png",

    "Grazioso_Salvare_Logo.png",

    "grazioso_logo.png",

    "/mnt/data/Grazioso Salvare Logo.png",

]

logo_path = next((p for p in logo_candidates if os.path.exists(p)), None)



brand_row = html.Div(

    [

        html.A(

            html.Img(src=app.get_asset_url(os.path.basename(logo_path))) if logo_path else html.Span(""),

            href="https://www.snhu.edu", target="_blank", style={"textDecoration": "none"}

        ) if logo_path else html.A("Grazioso Salvare", href="https://www.snhu.edu", target="_blank"),

        html.Div([html.Span("Dashboard by "), html.Strong("Jordan Bankston")],

                 style={"marginLeft":"16px","fontSize":"16px"}),

    ],

    style={"display":"flex","alignItems":"center","gap":"12px","marginBottom":"8px"}

)



filters = html.Div(

    [dcc.RadioItems(

        id="rescue-filter",

        options=[

            {"label":"Reset (All)","value":"Reset"},

            {"label":"Swift Water Team","value":"Water"},

            {"label":"Mountain Patrol","value":"Mountain"},

            {"label":"Urban Disaster Response","value":"Disaster"},

        ],

        value="Reset",

        labelStyle={"display":"inline-block","marginRight":"16px"}

    )],

    style={"padding":"8px 0","borderBottom":"1px solid #ddd","marginBottom":"8px"}

)



df_init = fetch_df({})

columns = [{"name":c,"id":c} for c in df_init.columns if c != "_id"]



data_table = dash_table.DataTable(

    id="animals-table",

    columns=columns,

    data=df_init.drop(columns=["_id"], errors="ignore").to_dict("records"),

    page_size=10,

    filter_action="native",

    sort_action="native",

    style_table={"overflowX":"auto"},

    style_cell={"fontFamily":"Arial","fontSize":12,"padding":"6px"},

    style_header={"fontWeight":"bold"},

)



def make_map_markers(df):

    pts = []

    if "location_lat" in df.columns and "location_long" in df.columns:

        sub = df.dropna(subset=["location_lat","location_long"])

        for _, r in sub.iterrows():

            try:

                lat = float(r["location_lat"]); lon = float(r["location_long"])

                label = f"{r.get('name','(no name)')} — {r.get('breed','?')}"

                pts.append(dl.Marker(position=[lat,lon], children=[dl.Tooltip(label)]))

            except Exception:

                pass

    return pts



map_component = dl.Map(

    id="geo-map",

    center=[30.27,-97.74],

    zoom=9,

    style={"width":"100%","height":"420px"},

    children=[dl.TileLayer(), dl.LayerGroup(id="markers")]

)



if not df_init.empty and "breed" in df_init.columns:

    fig_init = px.pie(df_init, names="breed", title="Breed Distribution (current filter)")

else:

    fig_init = px.pie(pd.DataFrame({"breed": [], "count": []}), names="breed", values="count",

                      title="Breed Distribution (current filter) — no data")

pie_graph = dcc.Graph(id="breed-pie", figure=fig_init)



app.layout = html.Div(

    [

        brand_row,

        filters,

        html.Div([html.Div([html.H4("Animals"), data_table], style={"flex":"1","minWidth":"480px"})],

                 style={"display":"flex","gap":"16px","flexWrap":"wrap"}),

        html.Div(

            [

                html.Div([html.H4("Geolocation"), map_component], style={"flex":"1","minWidth":"480px"}),

                html.Div([html.H4("Distribution"), pie_graph], style={"flex":"1","minWidth":"480px"}),

            ],

            style={"display":"flex","gap":"16px","marginTop":"16px","flexWrap":"wrap"}

        ),

        html.Div(id="debug-msg", style={"marginTop":"8px","color":"#555"})

    ],

    style={"padding":"12px"}

)


In [9]:


# --- Callbacks ---

@app.callback(

    Output("animals-table","data"),

    Output("animals-table","columns"),

    Output("markers","children"),

    Output("breed-pie","figure"),

    Output("debug-msg","children"),

    Input("rescue-filter","value"),

)

def update_dashboard(filter_value):

    if filter_value == "Reset":

        q = {}; label = "Reset (All)"

    else:

        q = build_filter(filter_value); label = filter_value



    df = fetch_df(q)

    cols = [{"name":c,"id":c} for c in df.columns if c != "_id"]

    table_data = df.drop(columns=["_id"], errors="ignore").to_dict("records")

    markers = make_map_markers(df)



    if "breed" in df.columns and not df.empty:

        fig = px.pie(df, names="breed", title=f"Breed Distribution — {label}")

    else:

        fig = px.pie(pd.DataFrame({"breed": [], "count": []}), names="breed", values="count",

                     title=f"Breed Distribution — {label} (no data)")



    debug = f"Applied filter: {label}. Returned {len(df)} records."

    return table_data, cols, markers, fig, debug


In [10]:


# --- Run in JupyterLab (opens a new tab called "Dash (port: PORT)") ---

print(f"Launching Dash in JupyterLab mode on proxied port {PORT} ...")

app.run_server(mode="jupyterlab", port=PORT, debug=False)


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Oct/2025 18:39:45] "GET /_alive_2c163266-416e-4598-96cc-95d2d480c0d5 HTTP/1.1" 200 -


Launching Dash in JupyterLab mode on proxied port 8051 ...
